In [1]:

import torch
from datetime import datetime
import os
import pandas as pd
from pytorchvideo.data.encoded_video import EncodedVideo
import csv
import numpy as np
from PIL import Image

import torch.optim as optim
import torch.nn as nn
import glob
# from tqdm.auto import tqdm
# import wandb


/u/ez9517/miniconda3/envs/ellie_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
1

In [ ]:
from typing import Dict
import json
import urllib
from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
    UniformCropVideo
) 

In [69]:
# Choose the `slowfast_r50` model 
model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50', pretrained=False)

Using cache found in /u/ez9517/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [ ]:
CONFIG = {
    "temp_folder": "/n/fs/visualai-scr/temp_LLP/ellie",
    "data_root": "/n/fs/visualai-scr/Data/Kinetics_cvf/frames",  # Root directory for video frames
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "metadata_dir": "/n/fs/visualai-scr/Data/Kinetics_cvf/raw",  # Directory containing CSV files
    "batch_size": 1024, #according to paper
    "learning_rate": 0.01,
    "num_epochs": 196,
}

# Set to GPU or CPU
model = model.train()
model = model.to(CONFIG['device'])

if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    model = nn.DataParallel(model)

In [ ]:
wandb.watch(model, log='all', log_freq = 100)

In [ ]:
#Download the class names
#json_url = "https://dl.fbaipublicfiles.com/pyslowfast/dataset/class_names/kinetics_classnames.json"
#json_filename = "kinetics_classnames.json"
#urllib.request.urlretrieve(json_url, json_filename)#except: urllib.request.urlretrieve(json_url, json_filename)

PermissionError: [Errno 13] Permission denied: 'kinetics_classnames.json'

In [5]:
json_filename = os.path.join(CONFIG["temp_folder"], "slowfast_kinetics", "kinetics_classnames.json")

with open(json_filename, "r") as f:
    kinetics_classnames = json.load(f)

# Create an id --> label name mapping
kinetics_id_to_classname = {}
for k, v in kinetics_classnames.items():
    kinetics_id_to_classname[v] = str(k).replace('"', "")

In [6]:
#Create a classname --> id mapping
kinetics_classname_to_id = {v: k for k, v in kinetics_id_to_classname.items()}
#print(kinetics_classname_to_id)

In [7]:
#Define input transforms
side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256

num_frames_fast = 32 #how many frames the model expects (for the fast branch)
num_frames_slow = 8
#sampling_rate = 2 #how fast we sample frame for the fast branch (every two frames)
frames_per_second = 30 #original fps of the video
#slowfast_alpha = 4

In [8]:
transform = Compose(
        [
            Lambda(lambda x: x/255.0), #Scale values to [0, 1]
            NormalizeVideo(mean, std), #Normalize each channel
            ShortSideScale( #Scale the short side of the video to be "side_size", preserve the aspect ratio
                size=side_size
            ),
            CenterCropVideo(crop_size), #Take the center crop of [256x256]
        ]
)

In [16]:
#Initiate the Wandb
run = wandb.init(
    project="Slowfast_Kinetics"
)

Error in callback <bound method _WandbInit._pre_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7f6f8fb2ef80>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f6f62485e10, raw_cell="#Initiate the Wandb
run = wandb.init(
    project=.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bneuronic.cs.princeton.edu/n/fs/visualai-scr/temp_LLP/ellie/slowfast_kinetics/slowfast.ipynb#X13sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

BrokenPipeError: [Errno 32] Broken pipe

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7f6f8fb2ef80>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f6f62487af0, execution_count=16 error_before_exec=None error_in_exec=[Errno 32] Broken pipe info=<ExecutionInfo object at 7f6f62485e10, raw_cell="#Initiate the Wandb
run = wandb.init(
    project=.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bneuronic.cs.princeton.edu/n/fs/visualai-scr/temp_LLP/ellie/slowfast_kinetics/slowfast.ipynb#X13sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
#AN EXAMPLEEE

#url_link = "https://dl.fbaipublicfiles.com/pytorchvideo/projects/archery.mp4"
#video_path = 'archery.mp4'
#try: urllib.URLopener().retrieve(url_link, video_path)
#except: urllib.request.urlretrieve(url_link, video_path)

In [ ]:
#AN EXAMPLEEE


# Select the duration of the clip to load by specifying the start and end duration
# The start_sec should correspond to where the action occurs in the video
#start_sec = 0
#end_sec = start_sec + clip_duration

# Initialize an EncodedVideo helper class and load the video
#video = EncodedVideo.from_path(video_path)

# Load the desired clip
#video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)

# Apply a transform to normalize the video input
#video_data = transform(video_data)

# Move the inputs to the desired device

#inputs = video_data["video"]
#inputs = [i.to(CONFIG["device"])[None, ...] for i in inputs]

#for i, tensor in enumerate(inputs):
    #print(f"Tensor {i} shape: {tensor.shape}")

Tensor 0 shape: torch.Size([1, 3, 8, 256, 256])
Tensor 1 shape: torch.Size([1, 3, 32, 256, 256])


In [ ]:
##AN EXAMPLEEE

# Pass the input clip through the model
#preds = model(inputs)

# Get the predicted classes
#post_act = torch.nn.Softmax(dim=1)
#preds = post_act(preds)
#pred_classes = preds.topk(k=5).indices[0]

# Map the predicted classes to the label names
#pred_class_names = [kinetics_id_to_classname[int(i)] for i in pred_classes]
#print("Top 5 predicted labels: %s" % ", ".join(pred_class_names))

Top 5 predicted labels: blowing nose, ironing, cutting pineapple, cartwheeling, sharpening knives


In [2]:
# ========== DATA PIPELINE ==========
#Given a row in csv, get the folder to the frames
#Example: /n/fs/visualai-scr/Data/Kinetics_cvf/frames/train/milking cow/

def path_to_frames_fn(row):
    """Map CSV row to video file path"""
    return os.path.join(
        CONFIG["data_root"], row['split'],row['label'],
        f"{row['youtube_id']}_{int(row['time_start']):06d}_{int(row['time_end']):06d}"
    )
    

In [ ]:
import json
class KineticsDataset2(torch.utils.data.Dataset):
    def __init__(self, csv_path, split, stride=2.0, max_videos=None):
        """
        Args:
            csv_path: Path to CSV file.
            split: Dataset split name (e.g. 'train', 'val').
            transform: Transform to apply to video clips.
            clip_duration: Duration of each clip in seconds.
            stride: How far apart to sample clips (in seconds).
            max_clips_per_video: If not None, randomly sample up to this many clips per video.
        """
        self.csv_path = csv_path
        self.split = split
    
        num_videos=0 #used to just read a select number of videos bc it takes too long :(

        # Read the CSV and create a list of samples
        # Each sample is a tuple of (path to frames folder, label)


        self.samples = [] # Will store (path to frames, label)
        self.frames_files = {} # Key: path to frames folder, Value: list of frame files

        if os.path.exists("frames_files.json"):
            with open("frames_files.json", "r") as f:
                self.frames_files = json.load(f)

        
        with open(csv_path, newline='', encoding='utf-8') as csvfile:
            reader = list(csv.DictReader(csvfile))
            for row in tqdm(reader):
                num_videos += 1
                if max_videos is not None and num_videos > max_videos:
                    break

                frames_path = path_to_frames_fn(row)
                if not os.path.exists(frames_path):
                    continue

                if frames_path not in self.frames_files:
                    self.frames_files[frames_path] = sorted(glob.glob(os.path.join(frames_path, '*.jpg')))

                label = row['label']

                self.samples.append((frames_path, label))
                #print("FRAMES PATH:", frames_path)

        with open("frames_files.json", "w") as f:
            json.dump(self.frames_files, f)
        
        print("Among {} files, we found {}".format(len(reader), len(self.samples)))
        #Among 219782 files, we found 218371

    def __len__(self):
        return len(self.samples)
    
    # Compute indices for 8 and 32 evenly spaced frames
    #Num frames = total number of frames, n = # of frames to sample (8 or 32)
    #Returns a list of indices to sample from the total number of frames
    def sample_indices(self, n, num_frames):
        if num_frames < n:
            raise ValueError(f"Requested {n} frames, but only {num_frames} available.")
        return [int(round(i * (num_frames - 1) / (n - 1))) for i in range(n)]
    
    #Given the path to the frames directory and a list of indicies, load the video frames
    #Returns a tensor of the video frames
    def load_video_frames(self, frames_dir, frame_files, indices):
        frames = []
        for i in indices:
            if(i >= len(frame_files)):
                print(f"Index {i} is out of bounds for frame_files with length {len(frame_files)}")
                continue
            frame_path = os.path.join(frames_dir, frame_files[i])
            img = Image.open(frame_path).convert('RGB')  # Load as RGB
            img_tensor = torch.from_numpy(np.array(img)).permute(2, 0, 1)  # [C, H, W]
            frames.append(img_tensor)
    
        video_tensor = torch.stack(frames, dim=1)  # [3, num_frames, H, W]
        return video_tensor
            

    def __getitem__(self, idx):
        #frames_dir, label = self.samples[idx]
        #print("HELOOOOOOOOOOOOOOOOOO:", frames_dir)

        
        #for f in os.listdir(frames_dir):
            #if f.endswith('.jpg'):
                #print(f)
        
        #frames_files = self.frames_files[frames_dir]
        total_frames = len(frames_files)

        if total_frames < num_frames_fast:
            print("Total frames in directory:", total_frames, "Frames dir", frames_dir)

        #print("num_frames fast:", num_frames_fast)
        #print("num_frames slow:", num_frames_slow)

        idx_fast = self.sample_indices(num_frames_fast, total_frames)
        idx_slow = self.sample_indices(num_frames_slow, total_frames)

        #Shape should be [3, 32, 256, 256] for the fast tensor
        #Shape should be [3, 8, 256, 256] for the slow tensor
        fast_tensor = self.load_video_frames(frames_dir, frames_files, idx_fast)
        slow_tensor = self.load_video_frames(frames_dir, frames_files, idx_slow)

        #print("Fast tensor shape:", fast_tensor.shape)
        #print("Slow tensor shape:", slow_tensor.shape)

        fast_tensor = transform(fast_tensor)
        slow_tensor = transform(slow_tensor)

        inputs=[slow_tensor, fast_tensor]

        return {
            "inputs": inputs, #a list of 2 tensors
            "label": kinetics_classname_to_id[label],
            "video_path": frames_dir,
        }
import time
st = time.time()
dummy = KineticsDataset2(
    csv_path=os.path.join(CONFIG["metadata_dir"], "train.csv"),
    split="train",
)

print(time.time() - st)

 50%|█████     | 110314/219782 [12:26<03:43, 489.34it/s] 

In [ ]:
#run later
#with open("num_frames.json", "w") as f:
    #json.dump(dummy.num_frames, f, indent=4)

In [66]:
# Create a dataset instance for training set
train_dataset = KineticsDataset2(
    csv_path=os.path.join(CONFIG['metadata_dir'], "train.csv"),
    split="train",
    max_videos=None
)

print(len(train_dataset))

  8%|▊         | 17885/219782 [01:34<17:47, 189.08it/s]


KeyboardInterrupt: 

In [60]:
my_train_dataloader = torch.utils.data.DataLoader(train_dataset, CONFIG['batch_size'], shuffle=True)

In [ ]:

# ========== TRAINING PIPELINE ==========

def train_model():
    # Initialize components
    optimizer = optim.SGD(model.parameters(), lr=CONFIG['learning_rate'])
    loss_fn = nn.CrossEntropyLoss(reduction='mean')

    num_samples = 0
    count = 0
    
    # Training loop
    for epoch in range(CONFIG['num_epochs']):
        model.train()
        train_loss = 0.0

        # Training phase
        for i, batch in enumerate(my_train_dataloader):
            num_samples = len(batch[0]["inputs"])  # Number of samples in the batch
            count += 1
            #inputs = batch["inputs"]).to(CONFIG['device']
                                         
            inputs = [torch.tensor(x).to(CONFIG['device']) for x in batch["inputs"]]

            labels = batch["label"].to(CONFIG['device'])

            
            #print("Input shape", inputs[0].shape, inputs[1].shape)
            #print("Labels shape", labels.shape)

            optimizer.zero_grad()
            outputs = model(inputs)
            #print("MODEL OUTPUTS SHAPE", outputs.shape)
            loss = loss_fn(outputs, labels) #the avg of the losses for the samples in the batch
            loss.backward()
            optimizer.step()
        
            train_loss += loss.item()

            predicted = torch.argmax(outputs, dim=1)
            correct = (predicted == labels).sum().item()
            accuracy = float(correct)/num_samples


            #Log performance every 20 batches
            if i % 20 == 0:
                now = datetime.now()
                print("Epoch {}, Batch {}, Loss {:.4f}, Correct {}, Total {}, Time {}".format(epoch, i, train_loss, correct, num_samples, now.strftime("%Y-%m-%d %H:%M:%S")))
                file_path = f"/n/fs/visualai-scr/temp_LLP/ellie/slowfast_kinetics/weights_{count:06d}.pth"
                torch.save(model.state_dict(), file_path)
                run.log({"epoch": epoch, "loss": loss, "accuracy": accuracy, "correct": correct})

In [62]:
if __name__ == "__main__":
    train_model()

Total frames in directory: 0 Frames dir /n/fs/visualai-scr/Data/Kinetics_cvf/frames/train/slacklining/BTXR0tSDdAg_000054_000064


ValueError: Requested 32 frames, but only 0 available.